In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START
from typing_extensions import TypedDict, List
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from supabase.client import create_client

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Step 1: Create a function for Deepseek Call

In [2]:
import requests
import os
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

def call_deepseek(question, context):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}"}
    ]
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 512
    }

    response = requests.post(DEEPSEEK_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


# Step 2: Load documents, Split, and Index.

In [3]:
url="https://www.healthline.com/health/bodyweight-workout"

loader = WebBaseLoader([url])
docs = loader.load()
splitter = RecursiveCharacterTextSplitter()
splits = splitter.split_documents(docs)
 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_18853/986501637.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 3: Vector Store
Alternatively, we could use a cloud-based vector database.

In [4]:
vectorstore_in_memory = FAISS.from_documents(splits, embeddings)

# Step 4: Retrieval

In [6]:
query = "upper body program"
top_k = 4
retrieved_docs = vectorstore_in_memory.similarity_search(query, k=top_k)

# Combine context
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

# Step 5: Generate

In [7]:
# Get answer from DeepSeek
answer = call_deepseek(query, context)
print("\n DeepSeek Answer:\n", answer)


 DeepSeek Answer:
 Here’s an **upper body-focused bodyweight workout program** designed to build strength and muscle. This routine is suitable for **intermediate to advanced** fitness levels but can be modified for beginners.  

---

### **Upper Body Bodyweight Circuit**  
**Instructions:**  
- Perform each exercise for **45–60 seconds** (or 8–15 reps).  
- Rest **30–60 seconds** between exercises.  
- Complete **2–3 rounds** of the circuit.  
- Do this routine **2–3 times per week** with rest days in between.  

#### **1. Pushup Variations**  
- **Standard Pushups**: Hands shoulder-width apart, core engaged.  
- **Wide-Grip Pushups**: Hands wider than shoulders to target the chest.  
- **Diamond Pushups**: Hands close together (thumbs/index fingers touching) to emphasize triceps.  

#### **2. Pike Pushups**  
- Start in a Downward Dog position (hips high, hands on the floor).  
- Lower your head toward the ground by bending your elbows, then push back up.  
- *Targets shoulders and t